In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,initializers
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
def model():
  input=layers.Input(shape=(128,128,3))
  initializer=initializers.HeNormal()

  #downsampling
  #layer 1
  a1=layers.Conv2D(64,3,padding='same',activation='relu',kernel_initializer=initializer)(input)
  a2=layers.Conv2D(64,3,padding='same',activation='relu',kernel_initializer=initializer)(a1)
  p1=layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(a2)
  #layer 2
  b1=layers.Conv2D(128,3,padding='same',activation='relu',kernel_initializer=initializer)(p1)
  b2=layers.Conv2D(128,3,padding='same',activation='relu',kernel_initializer=initializer)(b1)
  p2=layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(b2)
  #layer 3
  c1=layers.Conv2D(256,3,padding='same',activation='relu',kernel_initializer=initializer)(p2)
  c2=layers.Conv2D(256,3,padding='same',activation='relu',kernel_initializer=initializer)(c1)
  p3=layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(c2)
  #layer 4
  d1=layers.Conv2D(512,3,padding='same',activation='relu',kernel_initializer=initializer)(p3)
  d2=layers.Conv2D(512,3,padding='same',activation='relu',kernel_initializer=initializer)(d1)
  p4=layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(d2)

  #bottleneck layer
  #layer 5
  e1=layers.Conv2D(1024,3,padding='same',activation='relu',kernel_initializer=initializer)(p4)
  e2=layers.Conv2D(1024,3,padding='same',activation='relu',kernel_initializer=initializer)(e1)

  #upsampling
  #layer 6
  u1=layers.Conv2DTranspose(512,(2,2),strides=2)(e2)
  f1=layers.concatenate([u1,d2])
  f2=layers.Conv2D(512,3,padding='same',activation='relu',kernel_initializer=initializer)(f1)
  f3=layers.Conv2D(512,3,padding='same',activation='relu',kernel_initializer=initializer)(f2)
  #layer 7
  u2=layers.Conv2DTranspose(256,(2,2),strides=2)(f3)
  g1=layers.concatenate([u2,c2])
  g2=layers.Conv2D(256,3,padding='same',activation='relu',kernel_initializer=initializer)(g1)
  g3=layers.Conv2D(256,3,padding='same',activation='relu',kernel_initializer=initializer)(g2)
  #layer 8
  u3=layers.Conv2DTranspose(128,(2,2),strides=2)(g3)
  h1=layers.concatenate([u3,b2])
  h2=layers.Conv2D(128,3,padding='same',activation='relu',kernel_initializer=initializer)(h1)
  h3=layers.Conv2D(128,3,padding='same',activation='relu',kernel_initializer=initializer)(h2)
  #layer 9
  u4=layers.Conv2DTranspose(64,(2,2),strides=2)(h3)
  i1=layers.concatenate([u4,a2])
  i2=layers.Conv2D(64,3,padding='same',activation='relu',kernel_initializer=initializer)(i1)
  i3=layers.Conv2D(64,3,padding='same',activation='relu',kernel_initializer=initializer)(i2)

  output=layers.Conv2D(1,1,padding='same',activation='softmax')(i3)
  cnn=tf.keras.Model(input,output,name='U-Net')
  return cnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def rescale(img):
  imresize=img.resize((128,128))
  return imresize

In [ ]:
def augment(imgarray):
  fliplist=[]
  for img in imgarray:
    mirror=np.flip(img,axis=1)
    fliplist.append(mirror)
  imgarray.extend(fliplist)
  return imgarray

In [ ]:
import pathlib
data=pathlib.Path('/content/drive/MyDrive/img')
images=list(data.glob('*.jpg'))
image=[]
mask=[]
outdir='/content/drive/MyDrive/out/'
for path in images:
  l1=str(path).split('/')
  outfile=outdir+l1[len(l1)-1]
  img=Image.open(path).convert('RGB')
  img=rescale(img)
  imgarr=np.array(img)/255
  image.append(imgarr)
  img=Image.open(outfile).convert('L')
  img=rescale(img)
  imgarr=np.array(img)/255
  mask.append(imgarr)

In [ ]:
img1=Image.fromarray((image[1000]*255).astype('uint8'), 'RGB')
img1

In [ ]:
img2=Image.fromarray((mask[1000]*255).astype('uint8'),'L')
img2

In [ ]:
image=augment(image)
mask=augment(mask)

In [ ]:
trainimage=image[0:1000]
trainmask=mask[0:1000]

In [ ]:
unet=model()

In [ ]:
from tensorflow.keras import optimizers
optim=optimizers.Adam()
unet.compile(optimizer=optim,loss='binary_crossentropy',metrics='accuracy')

In [ ]:
inputs=np.array([x for x in trainimage])
outputs=np.array([x for x in trainmask])

In [ ]:
tf.config.run_functions_eagerly(True)
history=unet.fit(inputs,outputs,batch_size=50,epochs=5)

In [ ]:
unet.save('partly_trained.h5')

In [ ]:
from keras.models import load_model
unet=load_model('partly_trained.h5')

In [ ]:
trainimage=image[4000:5000]
trainmask=mask[4000:5000]
inputs=np.array([x for x in trainimage])
outputs=np.array([x for x in trainmask])

In [ ]:
history=unet.fit(inputs,outputs,batch_size=50,epochs=5)